This assignment is taken from Svetlana Lazebnik.

### Common imports

In [ ]:
%matplotlib inline
import os
import sys
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import time

### Provided functions
#### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

#### Plot the albedo and the surface norms. 

In [ ]:
def plot_albedo_and_surface_normals(albedo_image, surface_normals):
    """
    albedo_image: h x w matrix
    surface_normals: h x w x 3 matrix.
    """
    fig, axes = plt.subplots(1, 4, figsize=(10,2.5))
    ax = axes[0]
    ax.axis('off')
    ax.set_title('albedo')
    ax.imshow(albedo_image, cmap='gray')

    ax = axes[1]
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0], cmap='jet')
    ax = axes[2]
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1], cmap='jet')
    ax = axes[3]
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2], cmap='jet')

    fig.colorbar(im, ax=axes, fraction=0.02, aspect=15)

#### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])

def display_3d(albedo_image, height_map, elev=10, azim=80):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(elev, azim)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

---
### Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    f=255;
    New_img = (imarray - ambimage[:,:,np.newaxis])
    
    New_img=New_img/f

    processed_imarray = np.where(New_img>0, New_img, 0)

    return processed_imarray

In [ ]:
def computealbedo(computed,height, width):
    albedot = computetemp(computed) 
    return np.reshape(albedot,(height,width)) 
def computetemp(computed):
    return np.linalg.norm(computed, axis = 0)

In [ ]:
def surfacenormals(computed,height,width):
    surface_normals_temp = computed/computetemp(computed)
    surface_normals_final = np.transpose(surface_normals_temp, (1,0)) 
    return np.reshape(surface_normals_final, (height,width,3))

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    height,width,N=imarray.shape
    
    imarray = np.transpose( imarray, (2, 0, 1))
    imarray = np.reshape(imarray,(N, height*width))
    
    computed= np.linalg.lstsq(light_dirs,imarray, rcond= None)[0]
    
    albedo_image = computealbedo(computed,height,width)

    surface_normals = surfacenormals(computed,height,width)
    print("The mean=",np.mean(computed))
#     print("imarray dim=",imarray.shape)
#     print("light_dirs dim=",light_dirs.shape)
#     print("albedo_image dim=",albedo_image.shape)
#     print("surface normar dim=",surface_normals.shape)
    return albedo_image, surface_normals

In [ ]:
def integrationcolumn(surface_normals):
    in_dirx = surface_normals[:,:,0]/surface_normals[:,:,2]
    in_diry = surface_normals[:,:,1]/surface_normals[:,:,2]
    return np.cumsum(in_dirx, axis=1)[0] + np.cumsum(in_diry, axis=0)

In [ ]:
def integrationrow(surface_normals):
    in_dirx = surface_normals[:,:,0]/surface_normals[:,:,2]
    in_diry = surface_normals[:,:,1]/surface_normals[:,:,2]
    return np.cumsum(in_dirx, axis=1) + np.cumsum(in_diry, axis=0)[0]

In [ ]:
def integrationaverage(surface_normals):
    return (integrationcolumn(surface_normals)+integrationrow(surface_normals))/2

In [ ]:
def integrationrandom(surface_normals):
        itrarray = []
        
        in_dirx = surface_normals[:,:,0]/surface_normals[:,:,2]
        in_diry = surface_normals[:,:,1]/surface_normals[:,:,2]
        
        #varcheck = 10
        for jitr in range(10):
            xr=in_dirx.shape[0]
            xc=in_dirx.shape[1]
            g=np.zeros([xr, xc])
            itrarray.append(g)
        for row in range(xr):
            for col in range(xc):
                zeromat=np.ones(row)
                onemat= np.zeros(col)
                varpat = np.append(zeromat,onemat)
                for jitr in range(10):
                    
                    ccurent = col
                    rcurent = row
                    
                    np.random.shuffle(varpat)
                    for x in varpat:
                        if x != 1:
                            itrarray[jitr][row][col] = in_dirx[rcurent][ccurent]+itrarray[jitr][row][col]
                            ccurent -= 1
                        else:
                            itrarray[jitr][row][col] = in_diry[rcurent][ccurent]+itrarray[jitr][row][col]
                            rcurent -= 1
        result=np.sum(itrarray, axis=0)
        returnres=result/10
        return returnres

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    if integration_method == 'row':
        return integrationrow(surface_normals)
    elif integration_method == 'column':
        return integrationcolumn(surface_normals)
    elif integration_method == 'average':
        return integrationaverage(surface_normals)
    elif integration_method == 'random':
        
        return integrationrandom(surface_normals)

### Main function

In [ ]:
root_path = './croppedyale/'
subject_name = 'yaleB02'
integration_method = 'average'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                    64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)

height_map = get_surface(surface_normals, 'average')

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_albedo_and_surface_normals(albedo_image, surface_normals)

In [ ]:
start_time = time.time()
height_map = get_surface(surface_normals, 'average')
Average_time=time.time() - start_time
display_3d(albedo_image, height_map, 00,00);
display_3d(albedo_image, height_map)
display_3d(albedo_image, height_map,60,20);
display_3d(albedo_image, height_map,30,70);
display_3d(albedo_image, height_map,20,90);
display_3d(albedo_image, height_map,80,20);
display_3d(albedo_image, height_map,20,20);

In [ ]:
start_time = time.time()
height_map = get_surface(surface_normals, 'row')
Row_time=time.time() - start_time
display_3d(albedo_image, height_map, 00,00);
display_3d(albedo_image, height_map)
display_3d(albedo_image, height_map,60,20);
display_3d(albedo_image, height_map,30,70);
display_3d(albedo_image, height_map,20,90);
display_3d(albedo_image, height_map,80,20);
display_3d(albedo_image, height_map,20,20);

In [ ]:
start_time = time.time()
height_map = get_surface(surface_normals, 'column')
Column_time=time.time() - start_time
display_3d(albedo_image, height_map, 00,00);
display_3d(albedo_image, height_map)
display_3d(albedo_image, height_map,60,20);
display_3d(albedo_image, height_map,30,70);
display_3d(albedo_image, height_map,20,90);
display_3d(albedo_image, height_map,80,20);
display_3d(albedo_image, height_map,20,20);

In [ ]:
start_time = time.time()
height_map = get_surface(surface_normals, 'random')
Random_time=time.time() - start_time
display_3d(albedo_image, height_map, 00,00);
display_3d(albedo_image, height_map)
display_3d(albedo_image, height_map,60,20);
display_3d(albedo_image, height_map,30,70);
display_3d(albedo_image, height_map,20,90);
display_3d(albedo_image, height_map,80,20);
display_3d(albedo_image, height_map,20,20);

In [ ]:
print("Time Taken for the Various methods")
print("Row     ", Row_time)
print("Column  ", Column_time)
print("Average ", Average_time)
print("Random  ", Random_time)